### the randomized experiment (RCT/ AB test)

reminder:

correlation (more generally, *association*) is not causation

...unless...maybe.. it is.

we have a treatment/intervention $ X \in \{0,1\}$

and we have an outcome variable $ Y $

`avg(Y | X = 1) - avg(Y | X = 0) = avg treatment effect (on the treated) + bias`

for now, let's give hand-wavy definitions:

treatment effect = "avg difference in $Y$ when the treatment is given vs when it is not given"

bias = "avg difference in $Y$ if noone got the treatment"

the nice thing about AB tests, is that they are designed to eliminate bias

how?

bias can be eliminated through random assignment of $X$.


In [5]:
import pandas as pd
cats = pd.read_csv('../cookie_cats.csv')

# mobile gaming dataset
# quite large: tens of thousands of observations

# we have to make up a story about the experimental set up:

# we have a mobile game (a game you play on your cell phone)
# intervention/experimental condition/X - "the thing that's different between the two groups"
# group A sees the gate at level 30
# group B sees the gate at level 40
# we measure 3 outcomes as a function of the intervention:
# total number of games played from the moment the experiment was launched to 14 days after.
# and retention .. after 1 day and after 7 days.

# let's assume prior to the experiment ALL players saw the gate at level 30
# that would mean the experimental condition is "player sees the gate at level 40"

cats.head(3)

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False


quick review

pvalue what does it mean?

probability of type 1 error, i.e. false positive rate, i.e. probability of rejecting the null hypothesis when you should NOT have

pvalue = "in a world where there's no difference between group A and B, we should expect to see data that looks like this pvalue percentage of the time"

if you do 1,000,000 hypothesis tests and get 1,000,000 pvalues of .0499999 (which leads you to reject the null every time) you will have 50,000 false positives. 

In [6]:
# randomization - we have to take it for granted here.
# what data would we need to do the random assignment ourselves
# we want to make sure that "on average" the players in group A and B 
# had the same level of engagement before the experiment

# what hypothesis test should we use
# t-test, specifically two-sample unpaired independent t-test

# null hypothesis: mean(sum of game rounds for group A) - mean(sum of game rounds for group B) = 0
# alternative hypothesis: ~null hypothesis 
# mean(sum of game rounds for group A) - mean(sum of game rounds for group B) != 0

# significance threshold
# most people arbitrarily use .05
# if p(false positive) < .05 we will decide to kind of believe the result

gate_30 = cats.loc[cats['version'] == "gate_30","sum_gamerounds"] # group A
gate_40 = cats.loc[cats['version'] == "gate_40","sum_gamerounds"] # group B

from scipy.stats import ttest_ind
ttest_ind(gate_30,gate_40)

TtestResult(statistic=0.8910426211362967, pvalue=0.37290868247405207, df=90187.0)

interpretation:

we fail to reject the null hypothesis

let's check our intuition about hypothesis testing

we've done this experiment...and now we want to explain where the variability in the data comes from, and there's really only three options:

**option 1**: all the variability is due to the systematic effect of the treatment

`gate_30_gr = (120, 120, 120, 120, ....)`

`gate_40_gr = (119, 119, 119, 119, ....)`

as you can see, this is highly unlikely to be the case.

**option 2**: all the variability is due to randomness/bias (noise) 

**^^^ when we do a hypothesis test, we're testing this ^^^**

**option 3:** the variability is some combination of systematic effect and noise

**^^^ when we reject the null, we are choosing to believe this ^^^**


In [7]:
# next
# 7 day retention (rate, category) ~ version (categorical (gate_30,40))
# proportions z test
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
# prep the data
retention_gate_30 = cats.loc[cats['version'] == "gate_30","retention_7"] # group A
retention_gate_40 = cats.loc[cats['version'] == "gate_40","retention_7"] # group B
count = np.array([retention_gate_30.sum(),retention_gate_40.sum()])
nobs = np.array([retention_gate_30.shape[0],retention_gate_40.shape[0]])
# run the test
# wait a sec
# what are the null/alt hypotheses
# null: retention rate gate_30 = retention_rate gate_40
# alt:  retention rate gate_30 != retention_rate gate_40 (or just: ~null)

_, pval = proportions_ztest(count, nobs)
# print the result
print(f'{round(pval,3)}')


0.002


In [8]:
# the test only hints at the strength and direction of the effect
# we need to take a closer look
cats.groupby('version')['retention_7'].agg(['sum','mean'])

,sum,mean
version,,
gate_30,8502,0.190201
gate_40,8279,0.182000


there's usually more than one way to answer the same question

statistical methods are like little robots - some are designed for a single purpose

for example, we used a t-test to evaluate sum_gamerounds (numeric) ~ version (categorical)

are there other tools that we could apply. what about a regression?

`avg(sum_gamerounds | version) = beta0 + beta_version*version{1,0}`